In [1]:
# Log client
from pylabnet.utils.logging.logger import LogClient

# Hardware clients
from pylabnet.hardware.wavemeter import high_finesse_ws7

# GUI client
from pylabnet.gui.pyqt import external_gui

# Script
from pylabnet.scripts.wlm_monitor import WlmMonitor

# Pause, update servers
from pylabnet.core.generic_server import GenericServer
from pylabnet.scripts.pause_script import PauseService
from pylabnet.scripts.parameter_update import UpdateService

ModuleNotFoundError: No module named 'pylabnet.scripts.wlm_monitor'

# Connect to devices

In [2]:
# Connect to Wavemeter server
wavemeter_client = high_finesse_ws7.Client(host='localhost', port=5678)
wavemeter_client.connect()

0

In [3]:
# Instantiate monitor script
wlm_monitor = WlmMonitor(wavemeter_client)

# Start servers for remote access

In [4]:
# Pause server

# Instantiate logger
log_client = LogClient(
    host='localhost', 
    port=1234, 
    module_tag='Pause'
)

# Instantiate Server
pause_service = PauseService()
pause_service.assign_module(module=wlm_monitor)
pause_service.assign_logger(logger=log_client)
pause_server = GenericServer(
    host='localhost', 
    port=7, 
    service=pause_service
)

#Start server
pause_server.start()

# Update server
update_service = UpdateService()
update_service.assign_module(module=wlm_monitor)
update_service.assign_logger(logger=log_client)
update_server = GenericServer(
    host='localhost',
    port=897,
    service=update_service
)
update_server.start()

## Check external connections

In [5]:
print("Pause server clients: "+str(pause_server._server.clients))
print("Update server clients: "+str(update_server._server.clients))

Pause server clients: {<socket.socket fd=1312, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 7), raddr=('127.0.0.1', 65414)>}
Update server clients: {<socket.socket fd=1360, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 897), raddr=('127.0.0.1', 65415)>}


# GUI

In [28]:
# Instantiate GUI client
gui_client = external_gui.Client(host='localhost', port=9)
gui_client.connect()

0

# Set parameters

In [29]:
wlm_monitor.set_params(
    channels=[
        {"channel": 1, 
         "setpoint": 407.06592, 
         "lock": False
        }
    ],
    gui=gui_client,
    display_pts=5000
)

# Monitor wavelength

In [31]:
wlm_monitor.resume()